In [1]:
import pandas as pd
import numpy as np

In [2]:
# Import data sources - HPA - DAVID - ORA - DisGeNet
hpa_data = pd.read_csv('../Data/HPA/Raw/NOT.tsv', sep='\t').drop_duplicates(subset='Gene')
dgn_data = pd.read_csv('../Data/NIH/Unique Values Per Positive Label.csv')
dgn_labels = pd.read_csv('../Data/NIH/DisGeNet_Labels_ensembl_conversion.csv')
dgn_labels.columns = ['Gene','Ensembl','Score_gda']
print(len(hpa_data.index), len(dgn_data.index))

15013 2456


In [3]:
identifiers = [
    "Gene",
    "Ensembl"
]
discrete_features = [
    "Protein class",
    "Biological process",
    "Molecular function",
    "Disease involvement", # consider gettting rid of this feature
    "Subcellular location",
]
continuous_features = [
    "Tissue RNA - lung [NX]",
    "Single Cell Type RNA - Mucus-secreting cells [NX]"
]

# Create new DF filtering for features of interest
hpa_features = hpa_data.loc[:, hpa_data.columns.isin(identifiers+discrete_features+continuous_features)]

# Check NAs for each feature
print("Feature Sparsity:\n", hpa_features.isna().sum())

Feature Sparsity:
 Gene                                                     0
Ensembl                                                  0
Protein class                                            0
Biological process                                    6858
Molecular function                                    6651
Disease involvement                                  10548
Subcellular location                                  3666
Tissue RNA - lung [NX]                                   0
Single Cell Type RNA - Mucus-secreting cells [NX]        0
dtype: int64


In [4]:
# leftover from comparing label sets
t2 = pd.merge(hpa_data, dgn_labels[['Gene','Score_gda']], on='Gene', how='left')
t3 = pd.merge(hpa_data, dgn_labels[['Ensembl','Score_gda']], on='Ensembl', how='left')
t2_m = t2.loc[~t2.Score_gda.isna()].drop_duplicates(subset='Gene')[['Gene','Ensembl','Score_gda']]
t3_m = t3.loc[~t3.Score_gda.isna()].drop_duplicates(subset='Gene')[['Gene','Ensembl','Score_gda']]
extra = t3_m.loc[~t3_m.Ensembl.isin(t2_m.Ensembl)]
print(extra)

           Gene          Ensembl  Score_gda
3732      DPCR1  ENSG00000168631        0.1
5528       H1FX  ENSG00000184897        0.2
5722  HIST1H2BE  ENSG00000274290        0.3
7891     MT-CO2  ENSG00000198712        0.1


In [5]:
hpa_label_data = pd.merge(hpa_features, dgn_data[['Gene','Score_gda']], on='Gene', how='left')
print(len(hpa_label_data.loc[~hpa_label_data.Score_gda.isna()].index))

for index, row in extra.iterrows() :
    hpa_label_data.at[index, 'Score_gda'] = row.Score_gda
print(len(hpa_label_data.loc[~hpa_label_data.Score_gda.isna()].index))

hpa_label_data

1826
1830


,Gene,Ensembl,Protein class,Biological process,Molecular function,Disease involvement,Subcellular location,Tissue RNA - lung [NX],Single Cell Type RNA - Mucus-secreting cells [NX],Score_gda
0,A2M,ENSG00000175899,"Cancer-related genes, Candidate cardiovascular...",NaN,"Protease inhibitor, Serine protease inhibitor",Cancer-related genes,NaN,227.4,0.0,NaN
1,A4GALT,ENSG00000128274,"Enzymes, Predicted membrane proteins","Lipid biosynthesis, Lipid metabolism","Glycosyltransferase, Transferase",NaN,Mitochondria,14.0,0.0,NaN
2,AAAS,ENSG00000094914,"Disease related genes, Potential drug targets,...","mRNA transport, Protein transport, Translocati...",NaN,Disease mutation,"Nuclear membrane,Centrosome,Cytosol",13.4,11.5,NaN
3,AACS,ENSG00000081760,"Enzymes, Predicted membrane proteins","Fatty acid metabolism, Lipid metabolism",Ligase,NaN,Vesicles,4.9,20.1,NaN
4,AADAC,ENSG00000114771,"Enzymes, Predicted intracellular proteins",NaN,Hydrolase,NaN,NaN,1.7,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
15008,ZXDB,ENSG00000198455,"Predicted intracellular proteins, Transcriptio...","Transcription, Transcription regulation",Activator,NaN,Nucleoplasm,2.3,6.1,NaN
15009,ZXDC,ENSG00000070476,"Predicted intracellular proteins, Transcriptio...","Transcription, Transcription regulation",Activator,NaN,Nucleoli,10.6,5.9,NaN
15010,ZYG11B,ENSG00000162378,Predicted intracellular proteins,Ubl conjugation pathway,NaN,NaN,"Golgi apparatus,Intermediate filaments",7.7,10.9,NaN
15011,ZYX,ENSG00000159840,"Plasma proteins, Predicted intracellular proteins","Cell adhesion, Host-virus interaction",NaN,NaN,"Plasma membrane,Actin filaments,Focal adhesion...",52.8,19.9,NaN


In [6]:
# fill in na's in labels column and turn to 01
hpa_label_data.Score_gda = hpa_label_data.Score_gda.fillna(0)
hpa_label_data.Score_gda = hpa_label_data.Score_gda.apply(lambda x: 1 if x >= 0.05 else 0)
hpa_label_data.Score_gda.value_counts()

0    14613
1      400
Name: Score_gda, dtype: int64

In [7]:
# Standardize expression data
hpa_label_data["Tissue RNA - lung [NX]"] = (hpa_label_data["Tissue RNA - lung [NX]"] - hpa_label_data["Tissue RNA - lung [NX]"].mean()) / hpa_label_data["Tissue RNA - lung [NX]"].std()
col = "Single Cell Type RNA - Mucus-secreting cells [NX]"
hpa_label_data[col] = (hpa_label_data[col] - hpa_label_data[col].mean()) / hpa_label_data[col].std()


In [8]:
# Clean data
def explode(feature) :
    return feature.apply(lambda x: x.replace(' ', '').split(','))

hpa_clean = hpa_label_data.fillna('')
for ft in discrete_features :
    hpa_clean[ft] = explode(hpa_clean[ft])
hpa_clean

,Gene,Ensembl,Protein class,Biological process,Molecular function,Disease involvement,Subcellular location,Tissue RNA - lung [NX],Single Cell Type RNA - Mucus-secreting cells [NX],Score_gda
0,A2M,ENSG00000175899,"[Cancer-relatedgenes, Candidatecardiovasculard...",[],"[Proteaseinhibitor, Serineproteaseinhibitor]",[Cancer-relatedgenes],[],11.006867,-0.105024,0
1,A4GALT,ENSG00000128274,"[Enzymes, Predictedmembraneproteins]","[Lipidbiosynthesis, Lipidmetabolism]","[Glycosyltransferase, Transferase]",[],[Mitochondria],-0.000111,-0.105024,0
2,AAAS,ENSG00000094914,"[Diseaserelatedgenes, Potentialdrugtargets, Pr...","[mRNAtransport, Proteintransport, Translocatio...",[],[Diseasemutation],"[Nuclearmembrane, Centrosome, Cytosol]",-0.031058,-0.086940,0
3,AACS,ENSG00000081760,"[Enzymes, Predictedmembraneproteins]","[Fattyacidmetabolism, Lipidmetabolism]",[Ligase],[],[Vesicles],-0.469481,-0.073416,0
4,AADAC,ENSG00000114771,"[Enzymes, Predictedintracellularproteins]",[],[Hydrolase],[],[],-0.634534,-0.105024,0
...,...,...,...,...,...,...,...,...,...,...
15008,ZXDB,ENSG00000198455,"[Predictedintracellularproteins, Transcription...","[Transcription, Transcriptionregulation]",[Activator],[],[Nucleoplasm],-0.603586,-0.095432,0
15009,ZXDC,ENSG00000070476,"[Predictedintracellularproteins, Transcription...","[Transcription, Transcriptionregulation]",[Activator],[],[Nucleoli],-0.175480,-0.095746,0
15010,ZYG11B,ENSG00000162378,[Predictedintracellularproteins],[Ublconjugationpathway],[],[],"[Golgiapparatus, Intermediatefilaments]",-0.325059,-0.087883,0
15011,ZYX,ENSG00000159840,"[Plasmaproteins, Predictedintracellularproteins]","[Celladhesion, Host-virusinteraction]",[],[],"[Plasmamembrane, Actinfilaments, Focaladhesion...",2.001158,-0.073730,0


In [9]:
# Collect all discrete Features
protein_class = hpa_clean["Protein class"].explode().unique()
biological_process = hpa_clean["Biological process"].explode().unique()
molecular_function = hpa_clean["Molecular function"].explode().unique()
disease_involvement = hpa_clean["Disease involvement"].explode().unique()
subcellular_location = hpa_clean["Subcellular location"].explode().unique()
GO_features = np.concatenate([protein_class, biological_process, molecular_function, disease_involvement, subcellular_location])

In [10]:
# One hot encode discrete features
RowFeatures = pd.DataFrame(data = 0,index = hpa_clean['Gene'],columns=GO_features)
counter = 0
for index, row in RowFeatures.iterrows() :
    features = hpa_clean.loc[counter][['Protein class', 'Biological process', 'Molecular function', 'Disease involvement', 'Subcellular location']].to_list()
    flattened = [item for sublist in features for item in sublist if item]
    for t in flattened :
        row[t] = 1
    counter +=1
RowFeatures

,Cancer-relatedgenes,Candidatecardiovasculardiseasegenes,Plasmaproteins,Predictedsecretedproteins,Enzymes,Predictedmembraneproteins,Diseaserelatedgenes,Potentialdrugtargets,Predictedintracellularproteins,Transporters,...,Mitoticchromosome,Intermediatefilaments,Cytokineticbridge,Rods&Rings,Mitoticspindle,Aggresome,Lysosomes,Microtubuleends,Kinetochore,Cleavagefurrow
Gene,,,,,,,,,,,,,,,,,,,,,
A2M,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A4GALT,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAAS,0,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
AACS,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AADAC,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDB,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
ZXDC,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
ZYG11B,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


## Matrix Factorization + Optimization
Using SVD to factorize our discrete variables

In [11]:
from sklearn.decomposition import TruncatedSVD

#dictionary of data frames
feat_embeds = {}

#Create the different feature embeddings
# try: 30,40,50,60,70,80,90,100,110
for i in range(9) :
    n_comp = (i+3)*10
    svd = TruncatedSVD(n_components = n_comp)
    svdModel = svd.fit(RowFeatures)
    visits_emb = svdModel.transform(RowFeatures)
    visits_emb_df = pd.DataFrame(data=visits_emb, index=RowFeatures.index)
    feat_embeds[n_comp] = pd.merge(visits_emb_df, hpa_clean[identifiers+continuous_features+['Score_gda']], on='Gene',how='inner')
    
    print(n_comp)

30
40
50
60
70
80
90
100
110


In [14]:
from sklearn.metrics import classification_report

def eval_model(model, X, y):
    preds = model.predict(X)
    clf_report = classification_report(y, preds, labels=[0, 1], target_names=['negative', 'positive'], digits=2)
    print(clf_report)

In [27]:
# Negative sample
negIDs = hpa_label_data.loc[hpa_label_data.Score_gda == 0].sample(hpa_label_data.Score_gda.sum()).index
negIDs

Int64Index([ 6303, 13434,  7844, 13458, 11123,  5170,  2553,  4224,  4787,
             5202,
            ...
            11464, 14174,  7485, 13635,  4161,  6844,  3151,  6984,  3160,
             2851],
           dtype='int64', length=400)

In [28]:
for i in range(7,8):
    n_comp = (i+3)*10
    
    # Create Sample data
    data = feat_embeds[n_comp]
    data_sample = data.loc[ data.index.isin(negIDs) | data.Score_gda == 1]
    
    # separate features and labels
    node_feats = data_sample.iloc[:,1:-1]
    node_labels = data_sample.iloc[:,-1]
    
    # create train-test split
    from sklearn.model_selection import train_test_split
    test_size = 0.25

    X_train, X_test, y_train, y_test = train_test_split(node_feats, node_labels, test_size=test_size, shuffle=True, stratify=node_labels, random_state=360)
    # NOTE: train test split is shuffled and stratified across labels


    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import GridSearchCV

    # parameter grid to search over
    parameters = {
                    'n_estimators':[20, 50, 100, 150, 200],
                    'criterion':['gini'],
                    'max_depth':[1, 2, 3, 4, 5]
                 }

    # base random forest model
    rf = RandomForestClassifier(n_jobs=-1)

    # perform a gridsearch with 5-fold crossvalidation to find the best model
    rf_clf = GridSearchCV(rf, parameters, n_jobs=-1, refit=True, cv=5, return_train_score=True)

    rf_clf.fit(X_train, y_train)

    print('----------- Number of embeddings: ',n_comp)
    # show choice of parameters that yielded the best performance
    print('Best Parameters')
    print(rf_clf.best_params_)
    print('\n')

    # evaluate model
    print('Training Metrics')
    eval_model(rf_clf.best_estimator_, X_train, y_train)

    print()
    print('Testing Metrics')
    eval_model(rf_clf.best_estimator_, X_test, y_test)
    print('------------------------------------------')

----------- Number of embeddings:  30
Best Parameters
{'criterion': 'gini', 'max_depth': 3, 'n_estimators': 50}


Training Metrics
              precision    recall  f1-score   support

    negative       0.76      0.86      0.81       300
    positive       0.84      0.73      0.78       300

    accuracy                           0.80       600
   macro avg       0.80      0.80      0.80       600
weighted avg       0.80      0.80      0.80       600


Testing Metrics
              precision    recall  f1-score   support

    negative       0.66      0.76      0.71       100
    positive       0.72      0.61      0.66       100

    accuracy                           0.69       200
   macro avg       0.69      0.69      0.68       200
weighted avg       0.69      0.69      0.68       200

------------------------------------------
----------- Number of embeddings:  40
Best Parameters
{'criterion': 'gini', 'max_depth': 2, 'n_estimators': 50}


Training Metrics
              precision 

In [38]:
# quick cleanup before saving to csv
final_embed = feat_embeds[100]
cols = final_embed.columns.tolist()

# Replace Score_gda with Label
cols.pop(-1)
cols.append('Label')
final_embed.columns = cols

# Rearrange columns
cols = [cols[0]] + [cols[101]] + cols[1:101] + cols[102:104]
final_embed = final_embed[cols]
final_embed

,Ensembl,0,1,2,3,4,5,6,7,8,...,92,93,94,95,96,97,98,99,Tissue RNA - lung [NX],Single Cell Type RNA - Mucus-secreting cells [NX]
0,ENSG00000175899,0.443505,0.313574,0.430307,-0.285337,-0.351412,1.545059,-0.450936,-0.444200,-0.116774,...,0.033076,0.069726,0.159449,0.031158,0.087127,0.812502,0.652882,0.117809,11.006867,-0.105024
1,ENSG00000128274,0.532289,0.789331,-0.122159,0.019000,1.427162,-0.065659,0.001669,0.386983,-0.028900,...,0.283313,0.104073,0.122308,-0.010488,0.071484,-0.042168,0.138073,-0.061587,-0.000111,-0.105024
2,ENSG00000094914,1.605229,0.936681,0.345886,0.183699,-1.001764,-0.846346,0.550995,0.368205,0.202731,...,-0.193989,0.169840,-0.373577,0.174107,-0.118314,0.093455,0.209565,-0.057426,-0.031058,-0.086940
3,ENSG00000081760,0.467196,0.690415,-0.066723,0.335478,1.017124,0.029579,-0.131714,0.212387,0.075120,...,-0.350115,-0.241371,0.016416,-0.012377,0.039609,-0.034740,0.007967,0.039796,-0.469481,-0.073416
4,ENSG00000114771,0.956718,0.202132,-0.490047,-0.524460,0.467132,-0.207775,-0.494652,0.192615,0.077836,...,0.001733,-0.000615,-0.004378,-0.000040,0.002857,-0.010442,0.016493,0.005563,-0.634534,-0.105024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15008,ENSG00000198455,1.436813,-1.455513,0.510199,0.224208,0.323736,-0.171141,-0.060697,0.088626,0.103071,...,0.003771,0.063757,-0.002031,0.008038,-0.023358,0.013035,-0.001910,0.022492,-0.603586,-0.095432
15009,ENSG00000070476,1.140170,-1.162868,0.551409,0.127004,0.119115,-0.120434,-0.421260,0.798186,0.041917,...,0.005172,0.045257,0.005712,0.001190,-0.027927,0.016542,-0.008024,0.035935,-0.175480,-0.095746
15010,ENSG00000162378,0.782655,-0.057857,-0.397211,0.017498,-0.135123,-0.248882,-0.438176,0.113363,-0.295955,...,0.028633,-0.015439,-0.001466,-0.019047,-0.041910,0.019608,0.020722,-0.070376,-0.325059,-0.087883
15011,ENSG00000159840,1.042306,0.183392,-0.327697,0.065425,-0.459893,0.507171,-0.610148,0.067066,0.368216,...,-0.135105,0.034374,0.453396,0.068095,-0.162314,-0.099047,0.128805,-0.140204,2.001158,-0.073730


In [39]:
# Save HPA
final_embed.to_csv('../Data/Final/HPA_processed_features_v1.1.csv')

In [30]:
# Save Gene list
final_embed['Ensembl'].to_csv('../Data/Final/HPA_gene_list.tsv', sep='\t', index=False, header=False)